In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/univ/4/nlp/alpha

Mounted at /content/drive
/content/drive/My Drive/univ/4/nlp/alpha


In [4]:
import pandas as pd
import numpy as np
# CSV 파일 읽기
csv_file = 'AlphaMist_total_result.csv'
df_total = pd.read_csv(csv_file)
df_total['답변'] = df_total['답변'].str.extract(r'답변:\s*(.*)')
df_total['RAG  답변'] = df_total['RAG  답변'].str.extract(r'response:\s*(.*)')
# 새로운 DataFrame을 확인
print(df_total['답변'].head())
print(df_total['RAG  답변'].head())
base3_csv_file = 'baseline3_AlphaMist_result.csv'
df = pd.read_csv(base3_csv_file)
df['response']=df['response'].str.extract(r'답변:\s*(.*)')
print(df['response'].head())

0    피해자의 맥락에서 제공한 정보에 따르면 피고인은 폭행죄에 대해 형사처벌을 받을 수 ...
1    도로교통법 제44조 제1항에 따르면 자동차운전면허가 없는 사람이 도로에서 자동차를 ...
2                                                  NaN
3    위 상황은 여러 가지 법령에 따라 처벌을 받을 수 있습니다. 다음은 구체적인 내용입니다:
4    1. 예, 피해자의 죽음으로 인해 살인죄로 처벌받을 수 있습니다. 살인죄는 일반적으...
Name: 답변, dtype: object
0    1. 위증: 위 판례에서 설명한 것처럼 피고인은 자신의 기억에 반하는 허위의 진술을...
1    1. 위증: 위 판례에서 설명한 것처럼 피고인은 사고 후 증언에서 허위 진술을 한 ...
2    피해자를 살해한 것은 형법 29조(존속살인)와 형법 26조(사체은닉)에 따라 처벌될...
3    피고인이 자신의 증인인 B를 유인하여 위증을 저지른 행위는 형법 288조에 따라 처...
4                                                  NaN
Name: RAG  답변, dtype: object
0    위 판례에서는 피해자가 피고인에게 폭행당한 것으로 피고인이 폭행죄로 처벌받을 수 있...
1    피고인은 위증과 방조로 인해 처벌을 받을 수 있습니다. 위증은 법원 절차에서 거짓 ...
2    1. 존속살인: 형법 제250조 제2항에 따라 누군가 존속인을 살해하는 경우 처벌받...
3                                                  NaN
4    1. 제가 저지른 행위는 살인죄로 처벌받을 수 있습니다. 살인죄는 누군가를 죽이거나...
Name: response, dtype: object


In [5]:
print("답변 열의 유효한 값 개수:", df_total['답변'].notna().sum())
print("RAG  답변 열의 유효한 값 개수:", df_total['RAG  답변'].notna().sum())
print("response 열의 유효한 값 개수:", df['response'].notna().sum())
valid_indices = df[df_total['답변'].notna() & df_total['RAG  답변'].notna()&df['response'].notna()].index
print("유효한 값 개수:", len(valid_indices))

답변 열의 유효한 값 개수: 137
RAG  답변 열의 유효한 값 개수: 100
response 열의 유효한 값 개수: 124
유효한 값 개수: 62


In [9]:
#baseline 1 mistral
from metrics import exact_match_score, accuracy, f1
valid_indices = df[df_total['답변'].notna() & df_total['RAG  답변'].notna()&df['response'].notna()].index
preds = df_total.loc[valid_indices, '답변'].tolist()
ground_truths = df_total.loc[valid_indices, '법령의 적용'].tolist()
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

F1 Score: 2.326578043833071%


In [6]:
#baseline3-1 mistral
from metrics import exact_match_score, accuracy, f1
valid_indices = df[df_total['답변'].notna() & df_total['RAG  답변'].notna()&df['response'].notna()].index
preds = df.loc[valid_indices, 'response'].tolist()
ground_truths = df.loc[valid_indices, '쿼리 법령의 적용'].tolist()
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

F1 Score: 6.079482467451576%


In [7]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=25db4f79064de421775e1ad431afd7bdcc436a4864f4b793fce599a2063c96ab
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
#baseline3-1 mistral
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

BLEU Score: 5.164377842697283e-156%
BLEU-1 Score: 1.301811378551462%
BLEU-2 Score: 0.6178932617721834%
ROUGE-1 Score: 5.953241362943766%
ROUGE-2 Score: 1.3309773574395085%
ROUGE-L Score: 5.820218416485502%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [11]:
#baseline2-1 mistral
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
preds = df_total.loc[valid_indices, 'RAG  답변'].tolist()
ground_truths = df_total.loc[valid_indices, '법령의 적용'].tolist()
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")
bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

F1 Score: 3.4125223132182407%
BLEU Score: 5.478777123170413e-155%
BLEU-1 Score: 1.3336300917878159%
BLEU-2 Score: 0.6270184657111469%
ROUGE-1 Score: 8.338882774271939%
ROUGE-2 Score: 0.5309139784946236%
ROUGE-L Score: 8.338882774271939%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_